# Dependencies

In [1]:
import MURA
import cv2
import image_manipulation
from multiprocessing import Pool
from models import *
import numpy as np
import glob
import matplotlib.pyplot as plt
import preprocessing
import json
from os import path, mkdir
import re
from PIL import Image

import keras
from argparse import ArgumentParser
import tensorflow as tf
from sklearn import metrics

c:\Users\SKill\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model to Run
This section lets the user edit on what model parameters to run, the model directory and parameters should exist prior to changing the file path

In [2]:
# All editable variables
model_file_path = "models/VAE/dataset_2/300_64_4"

In [3]:
# Get model parameters
try:
    params = json.load(open(model_file_path + '/parameters.json'))

    # Model to use
    model_is_VAE = params['is_VAE']
    # Model parameters
    multiplier = params['multiplier']
    latent_size = params['latent_size']
    input_shape = params['input_shape']

    # Training parameters
    epochs = params['num_epochs']
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']

    # Dataset Path
    dataset_file_path = params['dataset_path']

except:
    raise Exception("No parameters.json file found in the model's directory.")

In [4]:
def normalize_image(image):
    image = image.astype(np.float32)
    image = image / 255.0
    return image


def denormalize_image(image):
    image = image * 255.0
    image = image.astype(np.uint8)
    return image

In [5]:
# each array contains the training, validation, and testing in order
image_datasets = {'train': [], 'valid': []}
for dataset_name in image_datasets.keys():
    for image_path in glob.glob(f'{dataset_file_path}/{dataset_name}/*.png'):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # image = normalize_image(image)
        image_datasets[dataset_name].append(image)
    image_datasets[dataset_name] = np.array(image_datasets[dataset_name])

# Model Training
This section creates and trains the model

In [6]:
if __name__ == "__main__":

    if model_is_VAE:
        model = VAE(False, input_shape, multiplier, latent_size)
    else:
        model = UPAE(True, input_shape, multiplier, latent_size)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.build(input_shape=(None,) + tuple(input_shape))

    model.compile(optimizer= optimizer, loss='mse'
                  ,metrics=[tf.keras.metrics.Accuracy()])
    
    model.summary()
    
    try:
        model.load_weights(model_file_path + "/model_weights")
        no_plots = True
    except:
        no_plots = False
        train_data = image_datasets['train']
        validation_data = image_datasets['valid']

        # Where images of each epoch will be saved
        save_directory = model_file_path + '/callback_images'
        save_callback = SaveImageCallback(validation_data, save_directory=save_directory, vae=model_is_VAE)

        EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='mse_loss', patience=30,
                                                                  restore_best_weights=True)

        history_train = model.fit(train_data, 
                                epochs=epochs, 
                                batch_size=batch_size,
                                validation_data=(validation_data, validation_data),
                                callbacks=[save_callback, EarlyStopping_callback])
        
        # Saving the model's history
        json.dump(history_train.history, open(model_file_path + '/history.json', 'w'))

        #Save weights
        model.save_weights(model_file_path + '/model_weights')
   
    

Model: "encoder_decoder"


c:\Users\SKill\anaconda3\envs\DL\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1024      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_1 (Batc  (None, 16, 16, 128)      512       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 16, 16, 128)       0         
          

# Creating of different Plots

In [10]:
# Create directory for saving images
save_directory = model_file_path + '/plots'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)


In [11]:
def create_plot(history, metric, title, save_name):
    x = range(50, len(history[metric]))
    plt.plot(x, history[metric][50:], label=metric)
    plt.plot(x, history[f'val_{metric}'][50:], label=f'val_{metric}')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.title(title)
    plt.legend()
    plt.savefig(f'{save_directory}/{save_name}', dpi=300)
    plt.close()

# Check if UPAE or not

if no_plots:
    pass
elif model_is_VAE:
    # Get history of data
    history = history_train.history
    # Create plot for mse_loss
    create_plot(history, 'mse_loss', 'MSE Loss per Epoch', 'mse_loss.png')
    # Create plot for reconstruction_loss
    create_plot(history, 'reconstruction_loss', 'Reconstruction Loss per Epoch', 'reconstruction_loss.png')
else:
    # Get history of data
    history = history_train.history
    # Create plot for mse_loss
    create_plot(history, 'mse_loss', 'MSE Loss per Epoch', 'mse_loss.png')
    # Create plot for total loss
    create_plot(history, 'total_loss', 'Total Loss per Epoch', 'total_loss.png')
    # Create plot for loss1
    create_plot(history, 'loss1', 'Loss 1 per Epoch', 'loss1.png')
    # Create plot for loss2
    create_plot(history, 'loss2', 'Loss 2 per Epoch', 'loss2.png')
    # Create plot for binary_crossentropy
#     create_plot(history, 'reconstruction_loss', 'Reconstruction Loss per Epoch', 'reconstruction_loss.png')
#     # Create plot for accuracy
#     create_plot(history, 'accuracy', 'Accuracy per Epoch', 'accuracy.png')
    

# Testing of the Model with the Test Set
This section tests the model with the current test set
TODO: 
- Get the label of each image in the test set
- Test the images
- Create Linear Regression for the abnormality score to get the threshold for determining abnormal or normal images

In [12]:
# Get labels of each image in the image_datasets['test']
test_images = []
labels = []
for image_path in glob.glob(f'{dataset_file_path}/test/*.png'):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # image = normalize_image(image)
    # Get if it contains positive or negative
    if 'positive' in image_path:
        test_images.append(image)
        labels.append(1)
    else:
        test_images.append(image)
        labels.append(0)
test_images = np.array(test_images)

In [13]:
history_valid = model.predict(test_images, batch_size=batch_size)

UnboundLocalError: local variable 'reconstructed_float32' referenced before assignment

In [ ]:
abnor_scores = history_valid[1]
#converting to an array of numbers instead of tensor
abnor_scores = [item.numpy() for item in abnor_scores if isinstance(item, tf.Tensor)]
abnor_scores = [float(item) for item in abnor_scores]

In [ ]:
abnor_scores

In [ ]:
auc = metrics.roc_auc_score(labels, abnor_scores)
fpr, tpr, thresholds = metrics.roc_curve(labels, abnor_scores)

In [ ]:
tpr

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(labels, abnor_scores)
idx = None
min_diff = float('inf')
for i in range(len(fpr)):
    fnr = 1 - tpr[i]
    diff = abs(fpr[i] - fnr)
    if diff <= 5e-3:
        idx = i
        break
    elif diff < min_diff:
        min_diff = diff
        idx = i
assert idx is not None
t = thresholds[idx]

In [ ]:
#using threshold t , will be used to classify abnormality scores as normal or abnormal in the y_pred array;
y_pred = np.zeros_like(labels)
y_pred[abnor_scores < t] = 0
y_pred[abnor_scores >= t] = 1


# getting metrics score using y_pred which is now either 0  or 1
pres = metrics.precision_score(labels, y_pred)
sens = metrics.recall_score(labels, y_pred, pos_label=1)
spec = metrics.recall_score(labels, y_pred, pos_label=0)
f1 = metrics.f1_score(labels, y_pred)
error_rate = fpr[idx]

metrics_dict = {
    "Error rate": error_rate,
    "Precision": pres,
    "Sensitivity": sens,
    "Specificity": spec,
    "F1 score": f1
}


# Print the metrics
print("Error rate: {}".format(error_rate))
print("Precision: {}".format(pres))
print("Sensitivity: {}".format(sens))
print("Specificity: {}".format(spec))
print("F1 score: {}".format(f1))

# Save the metrics to a JSON file
with open(f"{save_directory}/metrics.json", "w") as json_file:
    json.dump(metrics_dict, json_file)

In [ ]:
print(save_directory)

# Creating Plot for abnormality scores of normal and abnormal images

In [ ]:
# Histogram for Positives and Negatives of the testing set
positive = []
negative = []
for i in range(len(labels)):
    if labels[i] == 1:
        positive.append(abnor_scores[i])
    else:
        negative.append(abnor_scores[i])

plt.hist(negative, color='blue', label='negative', bins=20)
plt.hist(positive, color='red', label='positive', bins=20)

# Get x and y labels
plt.xlabel('Abnormality Score')
plt.ylabel('Number of Images')
plt.title("Abnormality Score and Correct Label Histogram")
plt.legend()
plt.savefig(f'{save_directory}/testing_abnormality_scores.png')

In [ ]:
positive = []
negative = []
for i in range(len(labels)):
    if abnor_scores[i] == 1:
        positive.append(abnor_scores[i])
    else:
        negative.append(abnor_scores[i])


plt.hist(positive, color='red', label='positive', bins=20)
plt.hist(negative, color='blue', label='negative', bins=20)

# Get x and y labels
plt.xlabel('Abnormality Score')
plt.ylabel('Number of Images')
plt.title("Abnormality Score and Prediction Label Histogram")
plt.legend()
plt.savefig(f'{save_directory}/testing_abnormality_scores_prediction.png')

# Saving of final reconstructed images 

In [ ]:
# Create directory in models folder for reconstructed images
dataset_name = dataset_file_path.split('/')[-1]
reconstructed_images_path = model_file_path + "/" + dataset_name
if not path.exists(reconstructed_images_path):
    mkdir(reconstructed_images_path)

In [ ]:
def save_compared_reconstructed_image(original_image, reconstructed_image, abnormality_score, file_path, image_index,
                                      label, variance = None):
    # Put Images in cv2
    original_image = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)
    reconstructed_image = cv2.cvtColor(reconstructed_image, cv2.COLOR_GRAY2BGR)
    # Convert reconstructed image to uint8
    reconstructed_image = reconstructed_image.astype(np.uint8)
    # Denormalize all images
    # original_image = denormalize_image(original_image)
    # reconstructed_image = denormalize_image(reconstructed_image)

    if variance is not None:
        variance_image = cv2.cvtColor(variance, cv2.COLOR_GRAY2BGR)
        variance_image = variance_image.astype(np.uint8)

        reconstructed_variance_sub = np.abs(reconstructed_image - variance_image)

        reconstruction_error = np.abs(original_image - reconstructed_variance_sub)
        
        # variance_image = denormalize_image(variance_image)
        concatenated_img = cv2.hconcat((original_image, reconstructed_image, variance_image, reconstructed_variance_sub, 
                                        reconstruction_error))
    else:
        reconstruction_error = np.abs(original_image - reconstructed_image)
        concatenated_img = cv2.hconcat((original_image, reconstructed_image, reconstruction_error))
    # Save the image
    cv2.imwrite(f'{file_path}/Image_{image_index}_{abnormality_score}_{label}.png', concatenated_img)

In [ ]:
for i in range(len(test_images)):
    if model_is_VAE:
        save_compared_reconstructed_image(test_images[i], np.array(history_valid[0][i]), history_valid[1][i],
                                      reconstructed_images_path, i, labels[i])
    else:
        save_compared_reconstructed_image(test_images[i], np.array(history_valid[0][i]), history_valid[1][i],
                                      reconstructed_images_path, i, labels[i],  np.array(history_valid[2][i]))


# GIF for Callback_images

In [ ]:
def extract_epoch(filename):
    match = re.search(r'epoch_(\d+)', filename)
    if match:
        return int(match.group(1))
    return 0

def create_gif(folder_path, output_path, duration=50):
    images = []
    
    # Get all image file names from the folder
    filenames = os.listdir(folder_path)
    filenames = sorted(filenames, key=lambda x: extract_epoch(x))

    for filename in filenames:
        if filename.endswith('.png') or filename.endswith('.jpg'):
            file_path = os.path.join(folder_path, filename)
            image = Image.open(file_path).convert('P')
            images.append(image)
    
    # Save the images as a GIF
    images[0].save(output_path, save_all=True, append_images=images[1:], optimize=False, duration=duration, loop=0)


In [ ]:
gif_folder_path = model_file_path + "/" + "callback_images"
image_folders = ['image0', 'image1', 'image2']

for i,folder in enumerate(image_folders):
    curr_folder = gif_folder_path + "/" + folder
    gif_output_path = gif_folder_path + "/" + f"image{i}.gif"
    print(f"Creating GIF for: image{i}")
    create_gif(curr_folder, gif_output_path)